#### 지금까지 배운 모든 내용을 가지고 tesnsorflow에서 제공하는 mnist 데이터를 가지고 학습

In [2]:
# 모듈 삽입

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# data loading 

mnist = input_data.read_data_sets('./data/mnist', one_hot = True)

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz


In [56]:
# 그래프 초기화

tf.reset_default_graph()

In [57]:
# Placeholder

X = tf.placeholder(shape = [None, 784], dtype=tf.float32)
Y = tf.placeholder(shape = [None, 10], dtype=tf.float32)
keep_rate = tf.placeholder(dtype=tf.float32)

#### Convolution layer 설정
#### 만약 데이터 전처리 및 test, train data 분할 과정이 필요하다면 이전에 완료해야 함!

In [58]:
# image data 설정

x_img = tf.reshape(X, [-1,28,28,1])

In [59]:
# filter

# width, height, depth: image와 맞춰줌, filter 개수: 임의 설정
F1 = tf.Variable(tf.random_normal([3,3,1,32]), name ='Filter1')
L1 = tf.nn.conv2d(x_img,
                  F1,
                  strides=[1,1,1,1],
                  padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1,
                    ksize=[1,2,2,1],
                    strides=[1,2,2,1],
                    padding='SAME')


F2 = tf.Variable(tf.random_normal([3,3,32,64]), name ='Filter2')
L2 = tf.nn.conv2d(L1,
                  F2,
                  strides=[1,1,1,1],
                  padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2,
                    ksize=[1,2,2,1],
                    strides=[1,2,2,1],
                    padding='SAME')
print(L2.shape)

(?, 7, 7, 64)


#### Multi-layer multinomial logistic regression 과정

In [60]:
# 위의 값을 2차원으로 변환

L2 = tf.reshape(L2, [-1, 7*7*64])
conv = L2
print(conv.shape)

(?, 3136)


In [ ]:
# multi-layer 설정

In [61]:
W1 = tf.get_variable("weight1", 
                     shape = [7*7*64,256],  #256은 임의의 logistic 노드 갯수
                     initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal(shape = [256]), name = 'bias1')

_layer1 = tf.nn.relu(tf.matmul(conv, W1) + b1)
layer1 = tf.nn.dropout(_layer1, keep_prob = keep_rate)  

# keep_prob: 노드를 살릴 확률
# 버전이 올라가면서 rate: 노드를 죽일 확률

In [62]:
W2 = tf.get_variable("weight2", 
                     shape = [256,10],  #최종 값은 label의 갯수: 10
                     initializer = tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal(shape = [10]), name = 'bias2')

In [63]:
# Hypothesis

logit = tf.matmul(layer1, W2) + b2
H = tf.nn.relu(logit)

In [64]:
# cost

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logit,
                                                                 labels = Y))

In [69]:
# Train

train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [70]:
# Session, 초기화

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [71]:
# 학습

num_of_epoch = 30
batch_size = 100

for step in range(num_of_epoch):
    num_of_iter = int(mnist.train.num_examples / batch_size)
    
    for i in range(num_of_iter):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        _, val_cost = sess.run([train, cost],
                               feed_dict = {X: batch_x,
                                            Y: batch_y,
                                            keep_rate: 0.7})
    if step % 1 == 0:
        print(f'cost값: {val_cost}')

    

cost값: 0.4190565049648285
cost값: 0.2415105104446411
cost값: 0.28697383403778076
cost값: 0.17715726792812347
cost값: 0.11836536228656769
cost값: 0.04389975592494011
cost값: 0.19270513951778412
cost값: 0.1926712989807129
cost값: 0.16013790667057037
cost값: 0.16698910295963287
cost값: 0.05905570834875107
cost값: 0.010394926182925701
cost값: 0.059021785855293274
cost값: 0.01400727964937687
cost값: 0.04886467754840851
cost값: 0.01665341481566429
cost값: 0.3109450340270996
cost값: 0.035629305988550186
cost값: 0.011684360913932323
cost값: 0.01017034612596035
cost값: 0.006488056853413582
cost값: 0.04088509455323219
cost값: 0.020686347037553787
cost값: 0.022999031469225883
cost값: 0.08725320547819138
cost값: 0.034166086465120316
cost값: 0.002041315659880638
cost값: 0.03405172377824783
cost값: 0.03294956311583519
cost값: 0.001109772827476263


In [73]:
# 정확도

predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32 ))

sess.run(accuracy, feed_dict = {X: mnist.test.images,
                                Y: mnist.test.labels,
                                keep_rate: 1})

0.9867